<a href="https://colab.research.google.com/github/jumbokh/nknu-class/blob/main/ML/notebooks/ch11_1_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# coding: utf-8
# # KNN
# * 加载数据集

In [3]:
%matplotlib inline
import os
import numpy as np
from scipy import stats
import pandas as pd
#import sklearn.cross_validation as cross_validation
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
orgData = pd.read_csv('date_data2.csv')
orgData.describe()

,income,attractive,assets,edueduclass,Dated,income_rank,attractive_rank,assets_rank
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,9010.000000,50.500000,96.006300,3.710000,0.500000,1.550000,1.560000,1.510000
std,5832.675288,28.810948,91.082226,1.225116,0.502519,1.140397,1.103896,1.123621
min,3000.000000,1.000000,3.728400,1.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.000000,28.000000,31.665269,3.000000,0.000000,1.000000,1.000000,0.750000
50%,7500.000000,51.000000,70.746924,4.000000,0.500000,2.000000,2.000000,2.000000
75%,11500.000000,68.875000,131.481061,4.000000,1.000000,3.000000,2.250000,2.250000
max,34000.000000,99.500000,486.311758,6.000000,1.000000,3.000000,3.000000,3.000000


In [6]:
# * 选取自变量

In [9]:
X = orgData.iloc[:, :4]
Y = orgData[['Dated']]
X.head()

,income,attractive,assets,edueduclass
0,3000,9.0,5.145476,1
1,3000,14.5,40.643781,4
2,3000,6.0,5.145476,1
3,3000,1.0,7.067434,1
4,3500,14.5,3.728400,2


In [ ]:
# * 极值标准化

In [10]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)
X_scaled[1:5]

array([[0.        , 0.13705584, 0.07649535, 0.6       ],
       [0.        , 0.05076142, 0.00293644, 0.        ],
       [0.        , 0.        , 0.00691908, 0.        ],
       [0.01612903, 0.13705584, 0.        , 0.2       ]])

In [ ]:
# * 划分训练集和测试集

In [12]:
train_data, test_data, train_target, test_target = train_test_split(
    X_scaled, Y, test_size=0.2, train_size=0.8, random_state=123)   #划分训练集和测试集

In [ ]:
# 上述过程有没有问题？

# * 建模

In [13]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)  # 默认欧氏距离
model.fit(train_data, train_target.values.flatten())
test_est = model.predict(test_data)

In [ ]:
# * 验证

In [14]:
import sklearn.metrics as metrics

print(metrics.confusion_matrix(test_target, test_est, labels=[0, 1]))  # 混淆矩阵
print(metrics.classification_report(test_target, test_est))

[[ 8  1]
 [ 1 10]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89         9
           1       0.91      0.91      0.91        11

    accuracy                           0.90        20
   macro avg       0.90      0.90      0.90        20
weighted avg       0.90      0.90      0.90        20



In [15]:
model.score(test_data, test_target)

0.9

In [ ]:
# * 选择k值

In [16]:
for k in range(1, 30):
    k_model = KNeighborsClassifier(n_neighbors=k)
    k_model.fit(train_data, train_target.values.flatten())
    score = k_model.score(test_data, test_target)
    print(k, '\t', score)

1 	 0.9
2 	 0.85
3 	 0.9
4 	 0.9
5 	 0.9
6 	 0.85
7 	 0.9
8 	 0.85
9 	 0.9
10 	 0.85
11 	 0.9
12 	 0.95
13 	 1.0
14 	 1.0
15 	 1.0
16 	 1.0
17 	 1.0
18 	 1.0
19 	 1.0
20 	 0.95
21 	 0.95
22 	 0.85
23 	 0.85
24 	 0.85
25 	 0.85
26 	 0.85
27 	 0.85
28 	 0.8
29 	 0.8


In [ ]:
# * 交叉验证选择k值

In [22]:
from sklearn import model_selection
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

n_samples = len(train_data)
kf = model_selection.KFold(n_splits=3)
grid = {'n_neighbors':[1,2,3,4,5,6,7,8,9]}
estimator = KNeighborsClassifier()
gridSearchCV = GridSearchCV(estimator, grid, cv=kf)
gridSearchCV.fit(train_data, train_target.values.flatten())
#gridSearchCV.grid_scores_
print(gridSearchCV.best_params_)

{'n_neighbors': 7}


In [23]:
best = gridSearchCV.best_estimator_ 
best.score(test_data, test_target)

0.9

In [ ]:
# 练习：试一试哪些参数会影响结果
###################################################################################################